# Wstęp

Metody uczenia maszynowego możemy podzielić na dwie główne kategorie (pomijając uczenie ze wzmocnieniem): nadzorowane i nienadzorowane. Uczenie **nadzorowane** (ang. *supervised*) to jest uczenie z dostępnymi etykietami dla danych wejściowych. Na parach danych uczących $dataset= \{(x_0,y_0), (x_1,y_1), \ldots, (x_n,y_n)\}$ model ma za zadanie nauczyć się funkcji $f: X \rightarrow Y$. Z kolei modele uczone w sposób **nienadzorowany** (ang. *unsupervised*) wykorzystują podczas trenowania dane nieetykietowane tzn. nie znamy $y$ z pary $(x, y)$.

Dość częstą sytuacją, z jaką mamy do czynienia, jest posiadanie małego podziobioru danych etykietowanych i dużego nieetykietowanych. Często annotacja danych wymaga ingerencji człowieka - ktoś musi określić co jest na obrazku, ktoś musi powiedzieć czy dane słowo jest rzeczownkiem czy czasownikiem itd.

Jeżeli mamy dane etykietowane do zadania uczenia nadzorowanego (np. klasyfikacja obrazka), ale także dużą ilość danych nieetykietowanych, to możemy wtedy zastosować techniki **uczenia częściowo nadzorowanego** (ang. *semi-supervised learning*). Te techniki najczęściej uczą się funkcji $f: X \rightarrow Y$, ale jednocześnie są w stanie wykorzystać informacje z danych nieetykietowanych do poprawienia działania modelu.

## Cel ćwiczenia

Celem ćwiczenia jest nauczenie modelu z wykorzystaniem danych etykietowanych i nieetykietowanych ze zbioru STL10 z użyciem metody [Bootstrap your own latent](https://arxiv.org/abs/2006.07733).

Metoda ta jest relatywnie "lekka" obliczeniowo, a także dość prosta do zrozumienia i zaimplementowania, dlatego też na niej się skupimy na tych laboratoriach.

# Zbiór STL10

Zbiór STL10 to zbiór stworzony i udostępniony przez Stanford [[strona]](https://ai.stanford.edu/~acoates/stl10/) [[papier]](https://cs.stanford.edu/~acoates/papers/coatesleeng_aistats_2011.pdf) a inspirowany przez CIFAR-10. Obrazy zostały pozyskane z [ImageNet](https://image-net.org/). Szczegóły można doczytać na ich stronie. To co jest ważne to to, że autorzy zbioru dostarczają predefiniowany plan eksperymentalny, żeby móc porównywać łatwo wyniki eksperymentów. Nie będziemy go tutaj stosować z uwagi na jego czasochłonność (10 foldów), ale warto pamiętać o tym, że często są z góry ustalone sposoby walidacji zaprojetowanych przez nas algorytmów na określonych zbiorach referencyjnych.

Korzystając z `torchvision.datasets` ***załaduj*** 3 podziały zbioru danych STL10: `train`, `test`, `unlabeled` oraz utwórz z nich instancje klasy `DataLoader`. Korzystając z Google Colab rozważ użycie Google Drive do przechowyania zbioru w calu zaoszczędzenia czasu na wielokrotne pobieranie.

In [1]:
import torchvision.datasets as datasets
import torch
from torch.utils.data import DataLoader
import os
from google.colab import drive
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm


drive.mount("/content/drive/")

path = '/content/drive/MyDrive/lab09/data/'
os.makedirs(path, exist_ok=True)

transform = transforms.Compose([transforms.ToTensor(),
                                    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))])


train_data = datasets.STL10(path, split='train', download=True, transform=transform)
test_data = datasets.STL10(path, split='test', download=True, transform=transform)
unlabeled_data = datasets.STL10(path, split='unlabeled', download=True, transform=transform)

# dataset_length = len(unlabeled_data)
# train_length = int(0.70 * dataset_length)
# val_length = dataset_length - train_length

# # Split the dataset
# unlabeled_data, val_dataset = torch.utils.data.random_split(unlabeled_data, [train_length, val_length])



batch_size = 32  # Możesz dostosować rozmiar partii do swoich potrzeb

train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
unlabeled_loader = DataLoader(unlabeled_data, batch_size=256, shuffle=True)





Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Uczenie nadzorowane

Żeby porównać czy metoda BYOL przynosi nam jakieś korzyści musimy wyznaczyć wartość bazową metryk(i) jakości, których będziemu używać (np. dokładność).

***Zaimplementuj*** wybraną metodę uczenia nadzorowanego na danych `train` z STL10. Możesz wykorzystać predefiniowane architektury w `torchvision.models` oraz kody źródłowe z poprzednich list.

In [ ]:
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim

resnet = models.resnet18(pretrained=True)



class SimpleCNNEncoder(torch.nn.Module):
    def __init__(self, repr=80):
        super().__init__()
        self.model = nn.Sequential(*list(resnet.children())[:-1])


        self.fc = nn.Sequential(
            nn.Linear(512, 256),  # Assuming input image size is 32x32
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, repr)
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x



class MyLinear(torch.nn.Module):
  def __init__(self, encoder):
    super().__init__()
    self.encoder = encoder

    self.fc = nn.Sequential(
        nn.Linear(128, 128),
        nn.ReLU(),
        nn.Linear(128, 10),

    )

    # for param in self.encoder.parameters():
    #     param.requires_grad = False

  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from typing import Tuple
from tqdm import tqdm


def count_correct(
    y_pred: torch.Tensor, y_true: torch.Tensor
) -> torch.Tensor:
    preds = torch.argmax(y_pred, dim=1)
    return (preds == y_true).float().sum()

def validate(
    model: nn.Module,
    loss_fn: torch.nn.CrossEntropyLoss,
    dataloader: DataLoader
) -> Tuple[torch.Tensor, torch.Tensor]:
    loss = 0
    correct = 0
    all = 0
    for X_batch, y_batch in dataloader:
        y_pred = model(X_batch.cuda())
        all += len(y_pred)
        loss += loss_fn(y_pred, y_batch.cuda()).sum()
        correct += count_correct(y_pred, y_batch.cuda())
    return loss / all, correct / all

def fit(
    model: nn.Module, optimiser: optim.Optimizer,
    loss_fn: torch.nn.CrossEntropyLoss,
    train_dl: DataLoader,
    val_dl: DataLoader,
    epochs: int,
    print_metrics: bool = False,
    name: str ='',
    patience: int = 3,
):
    last_better_epoch = 0
    low_val_loss = float('inf')
    low_acc_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        for X_batch, y_batch in tqdm(train_dl, desc=f"Epoch {epoch}"):
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            y_pred = model(X_batch)
            loss = loss_fn(y_pred, y_batch.long())

            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

        model.eval()
        with torch.no_grad():
            train_loss, train_acc = validate(model, loss_fn, train_dl)
            val_loss, val_acc = validate(model, loss_fn, val_dl)

            if val_loss < low_val_loss:
                low_val_loss = val_loss
                low_acc_loss = val_acc
                last_better_epoch = epoch

            if patience is not None and epoch - last_better_epoch > patience:
                print(f"Early stopping on epoch {epoch}")
                break

            # writer.add_scalars(
            #     main_tag='loss',
            #     tag_scalar_dict={
            #         f'train_{name}': train_loss,
            #         f'dev_{name}': val_loss
            #     },
            #     global_step=epoch+1
            # )

            # writer.add_scalars(
            #     main_tag=f'acc',
            #     tag_scalar_dict={
            #         f'train_{name}': train_acc,
            #         f'dev_{name}': val_acc
            #     },
            #     global_step=epoch+1
            # )
            if print_metrics:
                print(
                    f"Epoch {name} {epoch}: "
                    f"train loss = {train_loss:.3f} (acc: {train_acc:.3f}), "
                    f"validation loss = {val_loss:.3f} (acc: {val_acc:.3f})"
                )

    return low_val_loss, low_acc_loss

In [ ]:
model = MyLinear(SimpleCNNEncoder(repr=128)).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='vgg19')
print(result)

Epoch 0: 100%|██████████| 157/157 [00:05<00:00, 28.05it/s]


Epoch vgg19 0: train loss = 0.033 (acc: 0.618), validation loss = 0.039 (acc: 0.565)


Epoch 1: 100%|██████████| 157/157 [00:04<00:00, 32.95it/s]


Epoch vgg19 1: train loss = 0.021 (acc: 0.792), validation loss = 0.028 (acc: 0.703)


Epoch 2: 100%|██████████| 157/157 [00:04<00:00, 32.11it/s]


...

Epoch vgg19 3: train loss = 0.017 (acc: 0.836), validation loss = 0.034 (acc: 0.690)


Epoch 4: 100%|██████████| 157/157 [00:05<00:00, 28.71it/s]


Epoch vgg19 4: train loss = 0.012 (acc: 0.890), validation loss = 0.029 (acc: 0.718)


Epoch 5: 100%|██████████| 157/157 [00:04<00:00, 33.39it/s]


Early stopping on epoch 5
(tensor(0.0281, device='cuda:0'), tensor(0.7034, device='cuda:0'))


# Bootstrap your own latent

Metoda [Bootstrap your own latent](https://arxiv.org/abs/2006.07733) jest opisana w rodziale 3.1 papieru a także w dodatku A. Składa się z dwóch etapów:


1.   uczenia samonadzorowanego (ang. *self-supervised*)
2.   douczania nadzorowanego (ang. *fine-tuning*)

## Uczenie samonadzorowane

Architektura do nauczania samonadzorowanego składa się z dwóch sieci: (1) *online* i (2) *target*. W uproszczeniu cała architektura działa tak:


1.   Dla obrazka $x$ wygeneruj dwie różne augmentacje $v$ i $v'$ za pomocą funkcji $t$ i $t'$.
2.   Widok $v$ przekazujemy do sieci *online*, a $v'$ do *target*.
3.   Następnie widoki przekształacamy za pomocą sieci do uczenia reprezentacji (np. resnet18 lub resnet50) do reprezentacji $y_\theta$ i $y'_\xi$.
4.   Potem dokonujemy projekcji tych reprezentacji w celu zmniejszenia wymiarowości (np. za pomocą sieci MLP).
5.   Na sieci online dokonujmey dodatkowo predykcji pseudo-etykiety (ang. *pseudolabel*)
6.   Wyliczamy fukncję kosztu: MSE z wyjścia predyktora sieci *online* oraz wyjścia projekcji sieci *target* "przepuszczonej" przez predyktor sieci *online* **bez propagacji wstecznej** (*vide Algorithm 1* z papieru).
7.   Dokonujemy wstecznej propagacji **tylko** po sieci *online*.
8.   Aktualizujemy wagi sieci *target* sumując w ważony sposób wagi obu sieci $\xi = \tau\xi + (1 - \tau)\theta$ ($\tau$ jest hiperprametrem) - jest to ruchoma średnia wykładnicza (ang. *moving exponential average*).

Po zakończeniu procesu uczenia samonadzorowanego zostawiamy do douczania sieć kodera *online* $f_\theta$. Cała sieć *target* oraz warstwy do projekcji i predykcji w sieci *online* są "do wyrzucenia".

### Augmentacja

Dodatek B publikacji opisuje augmentacje zastosowane w metodzie BYOL. Zwróć uwagę na tabelę 6 w publikacji. `torchvision.transforms.RandomApply` może być pomocne.

***Zaimeplementuj*** augmentację $\tau$ i $\tau'$.


In [2]:
import random
from torch import nn
import torch
from torchvision import transforms as T


def get_default_aug() -> nn.Module:
    return torch.nn.Sequential(
        RandomApply(T.RandomRotation(degrees=(10, 60)), p=0.2).cuda(),
        T.RandomHorizontalFlip(),
        RandomApply(T.GaussianBlur((3, 3), (1.0, 2.0)), p=0.2).cuda(),
    )


class RandomApply(nn.Module):
    def __init__(self, fn: nn.Module, p: float):
        super().__init__()
        self.fn = fn
        self.p = p

    def forward(self, x):
        if random.random() > self.p:
            return x
        return self.fn(x)


### Implementacja uczenia samonadzorowanego

***Zaprogramuj*** proces uczenia samonadzorowanego na danych nieetykietowanych ze zbioru STL10.

Wskazówki do realizacji polecenia:

1. Proces uczenia może trwać bardzo długo dlatego zaleca się zastsowanie wczesnego zatrzymania lub uczenia przez tylko jedną epokę. Mimo wszystko powinno się dać osiągnąć poprawę w uczeniu nadzorowanym wykorzystując tylko zasoby z Google Colab.
2. Dobrze jest pominąć walidację na zbiorze treningowym i robić ją tylko na zbiorze walidacyjnym - zbiór treningowy jest ogromny i w związku z tym narzut czasowy na walidację też będzie duży.
3. Walidację modelu można przeprowadzić na zbiorze `train` lub całkowicie ją pominąć, jeżeli uczymy na stałej ilości epok.
4. Rozważ zastosowanie tylko jednej augmentacji - augmentacja $\tau'$ jest bardziej czasochłonna niż $\tau$.
5. Poniżej jest zaprezentowany zalążek kodu - jest on jedynie wskazówką i można na swój sposób zaimplementować tę metodę

In [11]:
import torch
from torch import nn, Tensor
import copy
import torch.nn.functional as F



resnet = models.resnet18(pretrained=True)




class SimpleCNNEncoder(torch.nn.Module):
    def __init__(self, repr=80):
        super().__init__()
        self.model = nn.Sequential(*list(resnet.children())[:-1])


        self.fc = nn.Sequential(
            nn.Linear(512, 256),  # Assuming input image size is 32x32
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, repr)
        )

    def forward(self, x):
        x = self.model(x)
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.fc(x)
        return x

class BYOL(nn.Module):
    def __init__(self, projection_size=256, labels_no=1000, tau=0.999):
        super().__init__()
        repr = 128
        self.online_encoder = SimpleCNNEncoder(repr=repr)
        self.online_projector = self.mlp(repr, projection_size)
        self.online_predictor = self.mlp(projection_size, labels_no)
        self.online_net = nn.Sequential(
            self.online_encoder,
            self.online_projector,
            self.online_predictor
        )

        self.target_encoder = self.copy_and_freeze_module(self.online_encoder)
        self.target_projector = self.copy_and_freeze_module(self.mlp(repr, labels_no))

        self.target_net = nn.Sequential(self.target_encoder, self.target_projector)
        for param in self.target_net.parameters():
            param.requires_grad = False

        self.aug_1 = get_default_aug()
        self.aug_2 = get_default_aug()

        self.tau = tau

    def forward(self, x: Tensor) -> tuple[Tensor, Tensor]:
        t = self.aug_1(x)
        t_prim = self.aug_2(x)

        q = self.online_net(t)
        q_sym = self.online_net(t_prim)

        with torch.no_grad():
            z_prim = self.target_net(t_prim)
            z_prim_sym = self.target_net(t)

        q = torch.cat([q, q_sym], dim=0)
        z_prim = torch.cat([z_prim, z_prim_sym], dim=0)

        return q, z_prim


    def mlp(self, encoder_out_shape, projection_size):
      return nn.Sequential(
          nn.Linear(encoder_out_shape, projection_size),
          nn.ReLU(),
          nn.BatchNorm1d(projection_size),
          nn.Linear(projection_size, projection_size)
      )

    def byol_loss(self, q: Tensor, z_prim: Tensor) -> Tensor:
        q_norm = F.normalize(q, dim=-1, p=2)
        z_prim_norm = F.normalize(z_prim, dim=-1, p=2)

        return( 2 - 2 * (q_norm * z_prim_norm).sum(dim=-1)).mean()

    @torch.no_grad()
    def update_target_network(self):
        for online_params, target_params in zip(self.online_net[0].parameters(), self.target_net[0].parameters()):
            target_params.data = target_params.data * self.tau + online_params.data * (1 - self.tau)


    def copy_and_freeze_module(self, model: nn.Module) -> nn.Module:
        copy_of_model = copy.deepcopy(model)
        for param in copy_of_model.parameters():
            param.requires_grad = False

        return copy_of_model


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [12]:
def fit_ssl(
    model: nn.Module, optimiser: optim.Optimizer,
    train_dl: DataLoader,
    epochs: int,
    print_metrics: bool = False,
    name: str ='',
):
    last_better_epoch = 0
    low_val_loss = float('inf')
    low_acc_loss = float('inf')
    for epoch in range(epochs):
        model.train()
        losses = []

        for X_batch, y_batch in tqdm(train_dl, desc=f"Epoch {epoch}"):
            X_batch = X_batch.to('cuda')
            y_batch = y_batch.to('cuda')
            q, z = model(X_batch)
            loss = model.byol_loss(q, z)
            losses.append(loss.item())

            loss.backward()
            optimiser.step()
            optimiser.zero_grad()

        model.update_target_network()
        model.eval()
        loss = sum(losses) / len(losses)

        if print_metrics:
            print(
                f"Epoch {name} {epoch}: "
                f"train loss = {loss} "
            )

    return low_val_loss, low_acc_loss

In [13]:
model_ssl = BYOL().cuda()
optimiser = optim.Adam(model_ssl.parameters(), lr=0.001)

result = fit_ssl(model_ssl, optimiser, unlabeled_loader, 4, print_metrics=True, name='byol')

Epoch 0: 100%|██████████| 391/391 [02:46<00:00,  2.35it/s]


Epoch byol 0: train loss = 1.5793217390089693 


Epoch 1: 100%|██████████| 391/391 [02:44<00:00,  2.38it/s]


Epoch byol 1: train loss = 1.5113740650284322 


Epoch 2: 100%|██████████| 391/391 [02:45<00:00,  2.37it/s]


Epoch byol 2: train loss = 1.482897459393572 


Epoch 3: 100%|██████████| 391/391 [02:44<00:00,  2.37it/s]

Epoch byol 3: train loss = 1.4636101558080414 


## Douczanie nadzorowane

***Zaimplementuj*** proces douczania kodera z poprzedniego polecenia na danych etykietowanych ze zbioru treningowego. Porównaj jakość tego modelu z modelem nauczonym tylko na danych etykietownaych. Postaraj się wyjaśnić różnice.

In [14]:
state_dict = model_ssl.online_encoder.state_dict()


In [15]:
encoder = SimpleCNNEncoder(repr=128)
encoder.load_state_dict(state_dict)

class MySSLinear(torch.nn.Module):
  def __init__(self, encoder):
    super().__init__()
    self.encoder = encoder

    self.fc = nn.Sequential(
        nn.Linear(128, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(128, 128),
        nn.BatchNorm1d(128),
        nn.ReLU(),
        nn.Linear(128, 10),
    )



  def forward(self, x):
    x = self.encoder(x)
    x = x.view(x.size(0), -1)
    x = self.fc(x)
    return x



In [19]:
model = MySSLinear(encoder).cuda()
optimiser = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='ft', patience=3)
print(result)

Epoch 0: 100%|██████████| 157/157 [00:06<00:00, 24.71it/s]


Epoch ft 0: train loss = 0.032 (acc: 0.671), validation loss = 0.036 (acc: 0.626)


Epoch 1: 100%|██████████| 157/157 [00:05<00:00, 26.95it/s]


Epoch ft 1: train loss = 0.018 (acc: 0.813), validation loss = 0.027 (acc: 0.710)


Epoch 2: 100%|██████████| 157/157 [00:05<00:00, 30.78it/s]


...

Epoch ft 6: train loss = 0.004 (acc: 0.959), validation loss = 0.025 (acc: 0.772)


Epoch 7: 100%|██████████| 157/157 [00:06<00:00, 25.52it/s]


Epoch ft 7: train loss = 0.006 (acc: 0.947), validation loss = 0.033 (acc: 0.743)


Epoch 8: 100%|██████████| 157/157 [00:05<00:00, 31.08it/s]


Early stopping on epoch 8
(tensor(0.0239, device='cuda:0'), tensor(0.7718, device='cuda:0'))


In [20]:
model = MySSLinear(encoder).cuda()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
loss_fn = nn.CrossEntropyLoss()

result = fit(model, optimiser, loss_fn, train_loader, test_loader, 50, print_metrics=True, name='ft', patience=3)
print(result)

Epoch 0: 100%|██████████| 157/157 [00:05<00:00, 30.95it/s]


Epoch ft 0: train loss = 0.036 (acc: 0.995), validation loss = 0.046 (acc: 0.809)


Epoch 1: 100%|██████████| 157/157 [00:06<00:00, 25.39it/s]


Epoch ft 1: train loss = 0.031 (acc: 0.999), validation loss = 0.042 (acc: 0.818)


Epoch 2: 100%|██████████| 157/157 [00:05<00:00, 28.52it/s]


...

Epoch ft 12: train loss = 0.028 (acc: 0.999), validation loss = 0.040 (acc: 0.816)


Epoch 13: 100%|██████████| 157/157 [00:05<00:00, 31.34it/s]


Epoch ft 13: train loss = 0.028 (acc: 0.999), validation loss = 0.039 (acc: 0.821)


Epoch 14: 100%|██████████| 157/157 [00:05<00:00, 26.22it/s]


Early stopping on epoch 14
(tensor(0.0388, device='cuda:0'), tensor(0.8255, device='cuda:0'))
